In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
from tqdm import tqdm

DATADIR = "/Users/dejitzen/Downloads/Datasets"

CATEGORIES = ["Dog", "Cat"]

for category in CATEGORIES:                
    path = os.path.join(DATADIR, category) # create path to dogs and cats
    for img in os.listdir(path): # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) # convert to array
        plt.imshow(img_array, cmap='gray') # graph it
        plt.show() # display!

        break
    break

- That's a 375 tall, 500 wide, and 3-channel image. 3-channel is because it's RGB (color)
- We definitely dont want the images that big,and also various images are different shapes and this is not good
- Lets Resize

In [ ]:
IMG_SIZE = 100

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array, cmap='gray')
plt.show()

- Next, we're going to want to create training data and we need to set aside some images for final testing
- Manually create a directory called Testing and then create 2 directories inside of there, one for Dog and one for Cat
- we are moving this images not copying them

In [ ]:
training_data = []

def create_training_data():
    for category in CATEGORIES: # do dogs and cats

        path = os.path.join(DATADIR, category) # create path to dogs and cats
        class_num = CATEGORIES.index(category) # get classifiation (0 or 1; 0=dog 1=cat)

        for img in tqdm(os.listdir(path)): # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) # convert to array
                new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE)) # resize to normalise data size
                training_data.append([new_array, class_num]) # add this to our training_data
            except Exception as e:
                pass

create_training_data()

In [ ]:
print(len(training_data))  

- We want to do is make sure our data is balanced
- If you do not balance, the model will initially learn that the best thing to do is predict only one class, whichever is the most common
- We want to shuffle the data

In [ ]:
import random

random.shuffle(training_data)

In [ ]:
for sample in training_data[:10]:
    print(sample[1])

Lets build out CNN Model

In [ ]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1)) 

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) # X has to be a numpy array, Keras doesnt accept anything else
y = np.array(y) # in the latest version, both X and y has to be Numpy/tensor objects


Lets save this data using Pickle, so we don't need to keep calculating everything we need to play with the neural network model
Python pickle, used for serializing and de-serializing objects into Python format text strings

In [ ]:
# import pickle

# pickle_out = open("X.pickle", "wb")
# pickle.dump(X, pickle_out)
# pickle_out.close()

# pickle_out = open("y.pickle", "wb")
# pickle.dump(y, pickle_out)
# pickle_out.close()

- The basic CNN structure is as follows: Convolution -> Pooling -> Convolution -> Pooling -> Fully Connected Layer -> Output

- Each convolution and pooling step is a hidden layer. After this, we have a fully connected layer, followed by the output layer. The fully connected layer is your typical neural network (multilayer perceptron) type of layer, and same with the output layer.

- Colors are coded as red, green and blue intensities in hexadecimal notation, where values is represented (0 - 255)

In [7]:
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten,Activation
from keras.callbacks import TensorBoard
import pickle
import time

NAME = "Cats-vs-dogs-CNN-64x2-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

X = X/255.0

model = Sequential()

#1st Layer
model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:]))     # grab the input shape dynamically 50x50x1 
model.add(Activation('relu'))                           # rectified linear activation function
model.add(MaxPooling2D(pool_size=(2,2)))

#2nd Layer
model.add(Conv2D(64, (3, 3)))  
model.add(Activation('relu'))                           # rectified linear activation function
model.add(MaxPooling2D(pool_size=(2,2)))

#3rd Layer
model.add(Flatten())           # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y, 
          batch_size=32, 
          epochs=6, 
          validation_split=0.3,
          callbacks=[tensorboard])


Epoch 1/6
546/546 [==============================] - 49s 89ms/step - loss: 0.6176 - accuracy: 0.6492 - val_loss: 0.5496 - val_accuracy: 0.7160
Epoch 2/6
546/546 [==============================] - 48s 88ms/step - loss: 0.5042 - accuracy: 0.7554 - val_loss: 0.5173 - val_accuracy: 0.7442
Epoch 3/6
546/546 [==============================] - 47s 86ms/step - loss: 0.4291 - accuracy: 0.7993 - val_loss: 0.4805 - val_accuracy: 0.7892
Epoch 4/6
546/546 [==============================] - 48s 89ms/step - loss: 0.3547 - accuracy: 0.8414 - val_loss: 0.4716 - val_accuracy: 0.7872
Epoch 5/6
546/546 [==============================] - 47s 87ms/step - loss: 0.2683 - accuracy: 0.8861 - val_loss: 0.4992 - val_accuracy: 0.7872
Epoch 6/6
546/546 [==============================] - 48s 89ms/step - loss: 0.1770 - accuracy: 0.9313 - val_loss: 0.6105 - val_accuracy: 0.7834
